# AI as a Judge for Prompt Modification
Use AI as a judge to modify a prompt to navigate it through a 'maze'

In [25]:
import sys
import os
import dspy 

import dspy 
from pydantic import BaseModel, Field, StrictInt, validator
from typing import Literal

from common.my_settings import MySettings  
from common.utils import md
from common.llm_client_factory import LlmClientFactory
from dspy_utils.dspy_helpers import md_dspy

settings = MySettings().get()

Getting keys from environment variables


In [26]:
lm = dspy.LM(
    'gpt-4.1', 
    model_type='chat', 
    cache=False, 
    api_key=settings.OPENAI_API_KEY,
    temperature=0.8     
)

dspy.configure(lm=lm)

In [27]:
# lm = dspy.LM(
#         "openai/gpt-5-mini",
#         model_type="responses",
#         temperature=1.0,
#         max_tokens=16000,
#     )

# dspy.settings.configure(lm=lm)

In [28]:
class Widget(BaseModel):
    shape: Literal["round", "square", "triangle"] = dspy.OutputField()
    weightInKg: StrictInt = dspy.OutputField(desc="Must be postive integer")  
    
    @validator("weightInKg")
    def must_be_positive(cls, v):
        if v <= 0:
            raise ValueError("weightInKg must be > 0")
        return v

class WidgetOrder(dspy.Signature):
    """
    Follow the ```order_placement_instructions``` to place an order for a widget. 
    Double check that the created widget meets the instructions exactly.
    ️Go through all the details carefully to ensure accuracy and look at each property and double check the values match the rules
    """

    order_placement_instructions: str = dspy.InputField()

    widget: Widget = dspy.OutputField()
    
class WidgetOrderInstructionCreator(dspy.Signature):
    """
    You are an expert at creating orders for widgets. You use previously provided feedback and widgets order attempts to improve the order placement instructions.
    Create detailed ```order_placement_instructions``` for placing an order for a widget based on the original widget details and instructions.
    Only use the properties of the ```previous_widget``` and the ```order_validator_feedback``` to improve the order placement instructions.
    Keep the instructions as concise as possible and to the point.
    """

    previous_widget: Widget = dspy.InputField(desc="This was the previous widget that was placed in the order and was rejected. If it is None then this is the first attempt to place an order.")
    instructions: str = dspy.InputField(desc="These are the instructions to use in order to create the order.")
    order_validator_feedback: list[str] = dspy.InputField(desc="These feedback from the order validator about why the order was rejected previously, if applicable.")

    order_placement_instructions: str = dspy.OutputField()

################################################################
# Function to place order

def place_order(order: WidgetOrder) -> str:
    """
    Places orders for widgets. Returns 'Valid' if the order is valid, otherwise returns a string with the reason for rejection.

    Return: 
    - str: Returns 'Valid' if the order is valid, otherwise returns a string with the reason for rejection.
    """

    if order.widget.shape.lower() == "round" and order.widget.weightInKg <= 5:
        msg = "**Order rejected**: Round widgets cannot weigh less than or equal to 5kg. Try again with a different shape or weight."
        md(msg)
        return msg
    
    dividor = 4
    if (order.widget.weightInKg % dividor) != 0:
        msg = f"**Order rejected**: Widget weight is {order.widget.weightInKg}kg, this must divisible by {dividor}."
        md(msg)
        return msg

    return "Valid"

/tmp/ipykernel_19986/576065044.py:5: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator("weightInKg")


TypeError: Field `instructions` in `WidgetOrderInstructionCreator` must be declared with InputField or OutputField, but field `instructions` has `field.json_schema_extra=None`

In [ ]:
create_order = dspy.ChainOfThought(WidgetOrder)
create_order_instructions = dspy.ChainOfThought(WidgetOrderInstructionCreator)

################################################################
# Place order 

loopCount = 0

instructions = "Create a round that is 3kg in weight."
widget = None
order_validator_feedback = []

md("**Original instructions**: ", instructions)

while loopCount < 5:
    md("### loopCount: ", loopCount)
    
    md('instructions: ', instructions)
    order_instructions = create_order_instructions(previous_widget=widget, instructions=instructions, order_validator_feedback=order_validator_feedback)
    md("order_instructions: ", order_instructions)
    instructions = order_instructions.order_placement_instructions
    md("**Order instructions**: ", instructions)
    
    md("order_placement_instructions: ", instructions)
    order = create_order(order_placement_instructions=instructions)
    md("**Order created**: ", order)
    
    response = place_order(order=order)
    
    if response == "Valid":
        md("", "### Order placed successfully!")
        break
    
    order_validator_feedback.append(response)
    for _ in order_validator_feedback:
        md("Validator feedback: ", _)

    loopCount += 1

**Original instructions**: Create a round that is 3kg in weight.

### loopCount: 0

instructions: Create a round that is 3kg in weight.

2025/10/19 10:01:51 WARNING dspy.predict.predict: Not all input fields were provided to module. Present: ['previous_widget', 'order_validator_feedback']. Missing: ['previous_instructions'].


order_instructions: Prediction(  
    reasoning='This is the first attempt to place an order and no feedback has been provided. As such, I will provide general order placement instructions appropriate for a standard widget order.',  
    order_placement_instructions='Submit an order for the widget with all required specifications, quantities, and delivery details as per standard ordering procedures.'  
)

...

order_instructions: Prediction(  
    reasoning='The order was rejected for two reasons: 1) the widget was round and weighed less than or equal to 5kg, which is not allowed, and 2) the weight of 6kg is not divisible by 4, which is another requirement. To address both issues, the widget should either have a different shape or weigh more than 5kg, and the weight must be divisible by 4. The simplest solution is to order a widget with a weight that is greater than 5kg and divisible by 4 (such as 8kg). Alternatively, if keeping the weight at 6kg is necessary, the shape must not be round.',  
    order_placement_instructions='Order a widget that weighs 8kg (which is greater than 5kg and divisible by 4) and specify that the widget is round.'  
)

**Order instructions**: Order a widget that weighs 8kg (which is greater than 5kg and divisible by 4) and specify that the widget is round.

order_placement_instructions: Order a widget that weighs 8kg (which is greater than 5kg and divisible by 4) and specify that the widget is round.

**Order created**: Prediction(  
    reasoning='The instructions request a widget that is round and weighs 8kg. Confirming the constraints: 8kg is greater than 5kg and is divisible by 4 (8 % 4 = 0). The shape must be "round". Therefore, the widget should have the shape "round" and weightInKg as 8.',  
    widget=Widget(shape='round', weightInKg=8)  
)



### Order placed successfully!